# **Out-of-Core Single-Cell Analysis with RAPIDS-SingleCell & Dask**  
**Author:** [Severin Dicks](https://github.com/Intron7)
**Copyright** [scverse](https://scverse.org)

In this notebook, we demonstrate the **out-of-core computation** capabilities of **rapids-singlecell** using **Dask**.  
This approach allows us to analyze **11 million cells** efficiently, even on relatively small hardware.  

By leveraging **Dask**, we can:  
- **Process large-scale single-cell datasets** without exceeding memory limits.  
- **Enable chunk-based execution**, loading only the necessary data into memory at any time.  

This method makes **large-scale single-cell analysis feasible** on standard hardware setups,  
removing barriers to working with massive datasets.


In [1]:
import dask
import time


from dask_cuda import LocalCUDACluster
from dask.distributed import Client

In [2]:
import rmm
import cupy as cp

from rmm.allocators.cupy import rmm_cupy_allocator

## **Initializing a Dask Cluster for Out-of-Core Computation**  

To enable **out-of-core computation** and parallel processing,  
we initialize a **Dask CUDA cluster**, which distributes computations across available GPU resources.  

We create a **local GPU cluster** with the following configuration:  


In [3]:
%%time
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES="0", threads_per_worker=30)

client = Client(cluster)

client

/home/intron/micromamba/envs/rapids-24.12/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44335 instead
  warnings.warn(


CPU times: user 309 ms, sys: 99.9 ms, total: 409 ms
Wall time: 2.46 s


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:44335/status,
Dashboard: http://127.0.0.1:44335/status,Workers: 1
Total threads: 30,Total memory: 62.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32909,Workers: 1
Dashboard: http://127.0.0.1:44335/status,Total threads: 30
Started: Just now,Total memory: 62.70 GiB
Comm: tcp://127.0.0.1:44863,Total threads: 30
Dashboard: http://127.0.0.1:45631/status,Memory: 62.70 GiB
Nanny: tcp://127.0.0.1:43137,


In [4]:
import rapids_singlecell as rsc
import anndata as ad

## **Loading Large Datasets into AnnData with Dask**  

To efficiently handle large-scale single-cell datasets, we load data directly from an **HDF5 (`h5`) or Zarr file**  
into an **AnnData object** using **Dask arrays**. This enables **lazy loading**, allowing data to be processed in chunks  
without exceeding memory limits.  

We achieve this using **`read_elem_as_dask`**, which loads the expression matrix (`X`) as a **Dask array**

In [7]:
from anndata.experimental import read_elem_as_dask

import h5py

SPARSE_CHUNK_SIZE = 20_000
data_pth = "h5/parse.h5ad"


f = h5py.File(data_pth)
X = f["X"]
shape = X.attrs["shape"]
adata = ad.AnnData(
    X = read_elem_as_dask(X, (SPARSE_CHUNK_SIZE, shape[1])),
    obs = ad.io.read_elem(f["obs"]),
    var = ad.io.read_elem(f["var"]))
f.close()



## **Transferring AnnData to GPU for Accelerated Computation**  

Once the dataset is loaded as a **Dask-backed AnnData object**,  
we transfer it to the **GPU** to leverage **RAPIDS-SingleCell’s** accelerated processing.  

We use **`rsc.get.anndata_to_GPU()`**, which efficiently moves the AnnData object to GPU memory:  


In [8]:
rsc.get.anndata_to_GPU(adata)

In [9]:
adata.X

dask.array<X_to_GPU, shape=(9697974, 40352), dtype=float32, chunksize=(20000, 40352), chunktype=cupyx.csr_matrix>

## **Quality Control (QC) Metrics Calculation (Requires Synchronization)**  

Before proceeding with further analysis, we compute **quality control (QC) metrics**  
to assess dataset quality and filter out low-quality cells or genes.  

We use **`rsc.pp.calculate_qc_metrics()`** to calculate key QC metrics

Although we are working with Dask-backed AnnData, this operation requires a synchronization step.
This means that Dask computations must be evaluated immediately,
so the process is not completely lazy like other out-of-core operations.

In [10]:
%%time
rsc.pp.calculate_qc_metrics(adata)

CPU times: user 7.13 s, sys: 1.75 s, total: 8.87 s
Wall time: 5min 6s


## **Filtering Cells and Genes Without Additional Computation**  

Instead of using **`sc.pp.filter_cells`** and **`sc.pp.filter_genes`**,  
we apply filtering directly using boolean indexing to **avoid extra computation**.

**Why Use Direct Indexing Instead of Built-in Functions?**
* More Efficient with Dask → Avoids triggering additional computations.
* Preserves Lazy Execution → Filtering is applied without forcing full dataset evaluation.
* Copy is Essential → Using `.copy()` prevents views, which may not work reliably with Dask-backed AnnData.

In [11]:
adata = adata[(adata.obs["n_genes_by_counts"]<=10000) 
            & (adata.obs["n_genes_by_counts"]>=200)].copy()
adata = adata[:,adata.var["n_cells_by_counts"]>=10].copy()

In [12]:
adata.X

dask.array<getitem, shape=(9697974, 40352), dtype=float32, chunksize=(20000, 40352), chunktype=cupyx.csr_matrix>

## **Log Normalization (Fully Lazy Execution)**  

Next, we apply **log normalization** to scale gene expression values.  
This step ensures that differences in sequencing depth across cells do not dominate downstream analysis.  

In [13]:
%%time
rsc.pp.normalize_total(adata,target_sum = 10000)
rsc.pp.log1p(adata)

CPU times: user 2.71 ms, sys: 0 ns, total: 2.71 ms
Wall time: 2.67 ms


## **Selecting Highly Variable Genes (Requires Synchronization)**  

To focus on the most informative features, we identify **highly variable genes (HVGs)**  
using the **Cell Ranger** method and subset the dataset accordingly.  

**Important Considerations:**
* Requires Synchronization → Computing highly variable genes triggers evaluation,
meaning this step is not fully lazy when using Dask.
* Copy is Essential → Using `.copy()` prevents views, ensuring the operation works properly with Dask-backed AnnData.


In [14]:
%%time
rsc.pp.highly_variable_genes(adata,n_top_genes=5000, flavor="cell_ranger")
adata = adata[:,adata.var.highly_variable].copy()

CPU times: user 7.26 s, sys: 1.8 s, total: 9.06 s
Wall time: 5min 7s


## **Scaling Gene Expression (Requires Synchronization)**  

To standardize gene expression values, we apply **feature scaling**,  
ensuring that all genes contribute equally to downstream analysis.

**Important Considerations:**
* Requires Synchronization → Since the input matrix is in CSR format (Compressed Sparse Row),
this step forces an immediate computation, meaning it is not fully lazy like earlier transformations.
* Scaling → Divides each gene’s expression values by its standard deviation.
* zero_center=False → Keeps the scaled values non-centered,
which is beneficial for sparse matrices and GPU acceleration.


In [15]:
%%time
rsc.pp.scale(adata, zero_center= False)

CPU times: user 6.37 s, sys: 1.83 s, total: 8.2 s
Wall time: 5min 9s


## **Principal Component Analysis (PCA) on GPU (Two-Step Synchronization Process)**  

To reduce dimensionality while preserving meaningful variation,  
we perform **Principal Component Analysis (PCA)** using **GPU acceleration**.

Understanding the Two-Step Synchronization in PCA:
1. Mandatory Synchronization for Covariance & Mean Calculation

    * PCA requires computing the covariance matrix and mean vector,
    which must be explicitly synchronized before proceeding.
    * This step is handled automatically within rsc.pp.pca().

2. Finalizing the Transformation with .compute()
    * After computing the principal components, the data remains lazy (Dask CuPy array).
    * Calling `.compute()` on `adata.obsm["X_pca"]` performs the final transformation,
      projecting the data onto the computed PCs and materializing the result as a fully computed CuPy array.

**Why This Matters?**
* The first synchronization (**covariance & mean**) ensures the PCA model is ready.
* The second synchronization (`compute()`) ensures that the transformed data is fully realized
for downstream analyses like clustering and UMAP.

In [16]:
%%time
rsc.pp.pca(adata, n_comps = 100,mask_var=None)
adata.obsm["X_pca"]=adata.obsm["X_pca"].compute()

/home/intron/micromamba/envs/rapids-24.12/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 37.29 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2025-02-21 13:52:01,358 - distributed.worker - ERROR - Compute Failed
Key:       ('getitem-01ef3937f344a54ca05bd1b0118bb5fc', 273, 0)
State:     executing
Task:  <Task ('getitem-01ef3937f344a54ca05bd1b0118bb5fc', 273, 0) _getitem(...)>
Exception: "MemoryError('std::bad_alloc: out_of_memory: CUDA error at: /tmp/pip-build-env-cl_nfrks/normal/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorMemoryAllocation out of memory')"
Traceback: '  File "/home/intron/micromamba/envs/rapids-24.12/lib/python3.12/site-packages/d

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /tmp/pip-build-env-cl_nfrks/normal/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorMemoryAllocation out of memory